In [36]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

conn = sqlite3.connect('example.db')
def rsq(sql):
    df = pd.read_sql(sql, conn)
    return df

In [97]:
df = pd.read_clipboard()
df.to_sql('match', conn, if_exists='append',index=False)
# df.to_sql('matches_italy', conn, if_exists='replace',index=False)

ParserError: Expected 7 fields in line 4, saw 8. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.

In [38]:
rsq("""SELECT
    -- Select the team long name and team API id
    team_long_name,
    team_api_id
FROM teams_germany
-- Only include FC Schalke 04 and FC Bayern Munich
WHERE team_long_name IN ('FC Schalke 04', 'FC Bayern Munich');
""")

,team_long_name,team_api_id
0,FC Bayern Munich,9823
1,FC Schalke 04,10189


In [39]:
rsq("""-- Identify the home team as Bayern Munich, Schalke 04, or neither
SELECT 
    CASE WHEN hometeam_id = 10189 THEN 'FC Schalke 04'
         WHEN hometeam_id = 9823 THEN 'FC Bayern Munich'
         ELSE 'Other' END AS home_team,
    COUNT(id) AS total_matches
FROM matches_germany
-- Group by the CASE statement alias
GROUP BY home_team;
""")

,home_team,total_matches
0,FC Bayern Munich,46
1,FC Schalke 04,47
2,Other,731


In [40]:
rsq("""SELECT 
    -- Select the date of the match
    date,
    -- Identify home wins, losses, or ties
    CASE WHEN home_goal > away_goal THEN 'Home win!'
         WHEN home_goal < away_goal THEN 'Home loss :(' 
         ELSE 'Tie' END AS outcome

FROM matches_spain;""")

,date,outcome
0,2012-01-21,Home loss :(
1,2012-01-22,Home win!
2,2012-01-22,Home loss :(
3,2012-01-23,Home win!
4,2012-01-21,Tie
...,...,...
375,2011-10-22,Home loss :(
376,2011-10-22,Tie
377,2011-10-23,Home win!
378,2011-10-23,Home loss :(


In [41]:
rsq("""SELECT 
    m.date,
    t.team_long_name AS opponent,
    -- Complete the CASE statement with an alias
    CASE WHEN m.home_goal > m.away_goal THEN 'Home win!'
         WHEN m.home_goal < m.away_goal THEN 'Home loss :(' 
         ELSE 'Tie' END AS outcome 
FROM matches_spain AS m
LEFT JOIN teams_spain AS t 
ON m.awayteam_id = t.team_api_id""")

,date,opponent,outcome
0,2012-01-21,Atlético Madrid,Home loss :(
1,2012-01-22,Athletic Club de Bilbao,Home win!
2,2012-01-22,FC Barcelona,Home loss :(
3,2012-01-23,Real Sporting de Gijón,Home win!
4,2012-01-21,Sevilla FC,Tie
...,...,...,...
375,2011-10-22,Real Madrid CF,Home loss :(
376,2011-10-22,Sevilla FC,Tie
377,2011-10-23,Real Zaragoza,Home win!
378,2011-10-23,Rayo Vallecano,Home loss :(


In [42]:
rsq("""SELECT 
    m.date,
    t.team_long_name AS opponent,
    -- Complete the CASE statement with an alias
    CASE WHEN m.home_goal > m.away_goal THEN 'Barcelona win!'
         WHEN m.home_goal < m.away_goal THEN 'Barcelona loss :(' 
         ELSE 'Tie' END AS outcome 
FROM matches_spain AS m
LEFT JOIN teams_spain AS t 
ON m.awayteam_id = t.team_api_id
-- Filter for Barcelona as the home team
WHERE m.hometeam_id = 8634; """)

,date,opponent,outcome
0,2011-10-29,RCD Mallorca,Barcelona win!
1,2011-11-19,Real Zaragoza,Barcelona win!
2,2011-12-03,Levante UD,Barcelona win!
3,2011-11-29,Rayo Vallecano,Barcelona win!
4,2012-01-15,Real Betis Balompié,Barcelona win!
5,2011-08-29,Villarreal CF,Barcelona win!
6,2012-05-02,Málaga CF,Barcelona win!
7,2012-02-04,Real Sociedad,Barcelona win!
8,2012-02-19,Valencia CF,Barcelona win!
9,2012-03-03,Real Sporting de Gijón,Barcelona win!


In [43]:
rsq("""-- Select matches where Barcelona was the away team
SELECT
    m.date,
    t.team_long_name AS opponent,
    CASE WHEN m.home_goal < m.away_goal THEN 'Barcelona win!'
         WHEN m.home_goal > m.away_goal THEN 'Barcelona loss :('
         ELSE 'Tie' END AS outcome
FROM matches_spain AS m
-- Join teams to matches
LEFT JOIN teams_spain AS t
ON m.hometeam_id = t.team_api_id
WHERE m.awayteam_id = 8634;""")

,date,opponent,outcome
0,2012-01-22,Málaga CF,Barcelona win!
1,2011-10-25,Granada CF,Barcelona win!
2,2011-11-06,Athletic Club de Bilbao,Tie
3,2011-11-26,Getafe CF,Barcelona loss :(
4,2011-12-10,Real Madrid CF,Barcelona win!
5,2012-01-08,RCD Espanyol,Tie
6,2012-01-28,Villarreal CF,Tie
7,2012-02-11,CA Osasuna,Barcelona loss :(
8,2012-02-26,Atlético Madrid,Barcelona win!
9,2012-03-11,Racing Santander,Barcelona win!


In [44]:
rsq("""SELECT 
    date,
    -- Identify the home team as Barcelona or Real Madrid
    CASE WHEN hometeam_id = 8634 THEN 'FC Barcelona' 
         ELSE 'Real Madrid CF' END AS home,
    -- Identify the away team as Barcelona or Real Madrid
    CASE WHEN awayteam_id = 8634 THEN 'FC Barcelona' 
         ELSE 'Real Madrid CF' END AS away
FROM matches_spain
WHERE (awayteam_id = 8634 OR hometeam_id = 8634)
      AND (awayteam_id = 8633 OR hometeam_id = 8633);""")

,date,home,away
0,2011-12-10,Real Madrid CF,FC Barcelona
1,2012-04-21,FC Barcelona,Real Madrid CF


In [47]:
rsq("""SELECT 
    date,
    CASE WHEN hometeam_id = 8634 THEN 'FC Barcelona' 
         ELSE 'Real Madrid CF' END as home,
    CASE WHEN awayteam_id = 8634 THEN 'FC Barcelona' 
         ELSE 'Real Madrid CF' END as away,
    -- Identify all possible match outcomes
    CASE WHEN home_goal > away_goal AND hometeam_id = 8634 THEN 'Barcelona win!'
        WHEN home_goal > away_goal AND hometeam_id = 8633 THEN 'Real Madrid win!'
        WHEN home_goal < away_goal AND awayteam_id = 8634 THEN 'Barcelona win!'
        WHEN home_goal < away_goal AND awayteam_id = 8633 THEN 'Real Madrid win!'
        ELSE 'Tie!' END as outcome
FROM matches_spain
WHERE (awayteam_id = 8634 OR hometeam_id = 8634)
      AND (awayteam_id = 8633 OR hometeam_id = 8633);""")

,date,home,away,outcome
0,2011-12-10,Real Madrid CF,FC Barcelona,Barcelona win!
1,2012-04-21,FC Barcelona,Real Madrid CF,Real Madrid win!


In [64]:
rsq("""-- Select the season and date columns
SELECT 
    season,
    date,
    -- Identify when Bologna won a match
    CASE WHEN hometeam_id = 9857 
          AND home_goal > away_goal 
         THEN 'Bologna Win'
         WHEN awayteam_id = 9857 
          AND away_goal > home_goal 
         THEN 'Bologna Win' 
          END AS outcome
-- select *
FROM matches_italy
-- limit 100 offset 500;""")

,season,date,outcome
0,2011/2012,2011-12-21,None
1,2011/2012,2011-12-21,None
2,2011/2012,2011-12-20,None
3,2011/2012,2011-12-21,None
4,2011/2012,2011-12-21,None
...,...,...,...
595,2012/2013,2013-03-30,None
596,2012/2013,2013-03-30,None
597,2012/2013,2013-03-30,None
598,2012/2013,2013-04-07,None


In [66]:
rsq("""-- Select the season, date, home_goal, and away_goal columns
SELECT 
    season,
    date,
    home_goal,
    away_goal
FROM matches_italy
WHERE
-- Exclude games not won by Bologna
    CASE WHEN hometeam_id = 9857 AND home_goal > away_goal THEN 'Bologna Win'
         WHEN awayteam_id = 9857 AND away_goal > home_goal THEN 'Bologna Win' 
         END IS NOT NULL;""")

,season,date,home_goal,away_goal
0,2011/2012,2011-10-30,3,1
1,2011/2012,2011-12-04,1,0
2,2011/2012,2012-01-08,2,0
3,2011/2012,2011-10-30,3,1
4,2011/2012,2011-12-04,1,0
5,2011/2012,2012-01-08,2,0
6,2011/2012,2011-10-30,3,1
7,2011/2012,2011-12-04,1,0
8,2011/2012,2012-01-08,2,0
9,2011/2012,2011-10-16,0,2


In [72]:
rsq("""SELECT 
    c.name AS country,
    -- Count matches in each of the 3 seasons
    COUNT(CASE WHEN m.season = '2012/2013' THEN m.id END) AS matches,
    COUNT(CASE WHEN m.season = '2013/2014' THEN m.id END) AS matches,
    COUNT(CASE WHEN m.season = '2014/2015' THEN m.id END) AS matches
FROM country AS c
LEFT JOIN match AS m
ON c.id = m.country_id
-- Group by country name alias
GROUP BY country;""")

,country,matches,matches,matches
0,Belgium,0,0,0
1,England,0,0,0
2,France,0,0,0
3,Germany,206,206,206
4,Italy,242,0,0
5,Netherlands,0,0,0
6,Poland,0,0,0
7,Portugal,0,0,0
8,Scotland,0,0,0
9,Spain,0,0,0


In [74]:
rsq("""SELECT 
    c.name AS country,
    -- Count matches in each of the 3 seasons
    COUNT(CASE WHEN m.season = '2012/2013' THEN m.id END) AS matches_2012_2013,
    COUNT(CASE WHEN m.season = '2013/2014' THEN m.id END) AS matches_2013_2014,
    COUNT(CASE WHEN m.season = '2014/2015' THEN m.id END) AS matches_2014_2015
FROM country AS c
LEFT JOIN match AS m
ON c.id = m.country_id
-- Group by country name alias
GROUP BY country;""")

,country,matches_2012_2013,matches_2013_2014,matches_2014_2015
0,Belgium,0,0,0
1,England,0,0,0
2,France,0,0,0
3,Germany,206,206,206
4,Italy,242,0,0
5,Netherlands,0,0,0
6,Poland,0,0,0
7,Portugal,0,0,0
8,Scotland,0,0,0
9,Spain,0,0,0


In [75]:
rsq("""SELECT 
    c.name AS country,
    -- Sum the total records in each season where the home team won
    SUM(CASE WHEN m.season = '2012/2013' AND m.home_goal > m.away_goal 
        THEN 1 ELSE 0 END) AS matches,
    SUM(CASE WHEN m.season = '2013/2014' AND m.home_goal > m.away_goal 
        THEN 1 ELSE 0 END) AS matches,
    SUM(CASE WHEN m.season = '2014/2015' AND m.home_goal > m.away_goal 
        THEN 1 ELSE 0 END) AS matches
FROM country AS c
LEFT JOIN match AS m
ON c.id = m.country_id
-- Group by country name alias
GROUP BY country;""")

,country,matches,matches,matches
0,Belgium,0,0,0
1,England,0,0,0
2,France,0,0,0
3,Germany,90,87,94
4,Italy,120,0,0
5,Netherlands,0,0,0
6,Poland,0,0,0
7,Portugal,0,0,0
8,Scotland,0,0,0
9,Spain,0,0,0


In [77]:
rsq("""SELECT 
    c.name AS country,
    -- Sum the total records in each season where the home team won
    SUM(CASE WHEN m.season = '2012/2013' AND m.home_goal > m.away_goal 
        THEN 1 ELSE 0 END) AS matches_2012_2013,
    SUM(CASE WHEN m.season = '2013/2014' AND m.home_goal > m.away_goal 
        THEN 1 ELSE 0 END) AS matches_2013_2014,
    SUM(CASE WHEN m.season = '2014/2015' AND m.home_goal > m.away_goal 
        THEN 1 ELSE 0 END) AS matches_2014_2015
FROM country AS c
LEFT JOIN match AS m
ON c.id = m.country_id
-- Group by country name alias
GROUP BY country;""")

,country,matches_2012_2013,matches_2013_2014,matches_2014_2015
0,Belgium,0,0,0
1,England,0,0,0
2,France,0,0,0
3,Germany,90,87,94
4,Italy,120,0,0
5,Netherlands,0,0,0
6,Poland,0,0,0
7,Portugal,0,0,0
8,Scotland,0,0,0
9,Spain,0,0,0


In [79]:
rsq("""SELECT 
    c.name AS country,
    -- Sum the home wins, away wins, and ties in each country
    COUNT(CASE WHEN m.home_goal > m.away_goal THEN m.id          
        END) AS home_wins,
    COUNT(CASE WHEN m.home_goal < m.away_goal THEN m.id 
        END) AS away_wins,
    COUNT(CASE WHEN m.home_goal = m.away_goal THEN m.id 
        END) AS ties
FROM country AS c
LEFT JOIN match AS m
ON c.id = m.country_id
GROUP BY country;""")

,country,home_wins,away_wins,ties
0,Belgium,0,0,0
1,England,0,0,0
2,France,0,0,0
3,Germany,364,249,211
4,Italy,193,103,103
5,Netherlands,0,0,0
6,Poland,0,0,0
7,Portugal,0,0,0
8,Scotland,0,0,0
9,Spain,188,98,94


In [81]:
rsq("""SELECT 
    c.name AS country,
    -- Calculate the percentage of tied games in each season
    AVG(CASE WHEN m.season= '2013/2014' AND m.home_goal = m.away_goal THEN 1
             WHEN m.season= '2013/2014' AND m.home_goal != m.away_goal THEN 0
             END) AS ties_2013_2014,
    AVG(CASE WHEN m.season= '2014/2015' AND m.home_goal = m.away_goal THEN 1
             WHEN m.season= '2014/2015' AND m.home_goal != m.away_goal THEN 0
             END) AS ties_2014_2015
FROM country AS c
LEFT JOIN match AS m
ON c.id = m.country_id
GROUP BY country;""")

,country,ties_2013_2014,ties_2014_2015
0,Belgium,NaN,NaN
1,England,NaN,NaN
2,France,NaN,NaN
3,Germany,0.223301,0.26699
4,Italy,NaN,NaN
5,Netherlands,NaN,NaN
6,Poland,NaN,NaN
7,Portugal,NaN,NaN
8,Scotland,NaN,NaN
9,Spain,NaN,NaN


In [83]:
rsq("""SELECT 
    c.name AS country,
    -- Round the percentage of tied games to 2 decimal points
    ROUND(AVG(CASE WHEN m.season='2013/2014' AND m.home_goal = m.away_goal THEN 1
             WHEN m.season='2013/2014' AND m.home_goal != m.away_goal THEN 0
             END),2) AS pct_ties_2013_2014,
    ROUND(AVG(CASE WHEN m.season='2014/2015' AND m.home_goal = m.away_goal THEN 1
             WHEN m.season='2014/2015' AND m.home_goal != m.away_goal THEN 0
             END),2) AS pct_ties_2014_2015
FROM country AS c
LEFT JOIN match AS m
ON c.id = m.country_id
GROUP BY country;""")

,country,pct_ties_2013_2014,pct_ties_2014_2015
0,Belgium,NaN,NaN
1,England,NaN,NaN
2,France,NaN,NaN
3,Germany,0.22,0.27
4,Italy,NaN,NaN
5,Netherlands,NaN,NaN
6,Poland,NaN,NaN
7,Portugal,NaN,NaN
8,Scotland,NaN,NaN
9,Spain,NaN,NaN
